# Динамическая память

## Зачем нужна динамическая память?

∙Стек программы ограничен. Он не предназначен для хранения больших объемов данных.

    // Не умещается на стек
    double m[10000000] = {}; // 80 Mb

∙Время жизни локальных переменных ограничено временем работы функции.  
∙Динамическая память выделяется в сегменте данных.  
∙Структура, отвечающая за выделение дополнительной памяти, называется кучей (не нужно путать содноимённой структурой данных).  
∙Выделение и освобождение памяти управляется вручную.

## Выделение памяти в стиле C

∙Стандартная библиотека cstdlib предоставляет четыре функции для управления памятью:

    void * malloc (size_t  size);
    void    free    (void * ptr);
    void * calloc (size_t  nmemb , size_t  size);
    void * realloc(void * ptr , size_t  size);

∙size_t — специальный целочисленный беззнаковый тип, может вместить в себя размер любого типа в байтах.  
∙Тип size_t используется для указания размеров типов данных, для индексации массивов и пр.  
∙void \* — это указатель на нетипизированную память (раньше для этого использовалось char *).

∙malloc — выделяет область памяти размера ≥ size. Данные не инициализируются.  
∙calloc — выделяет массив из nmemb размера size. Данные инициализируются нулём.  
∙realloc — изменяет размер области памяти по указателю ptr на size (если возможно, то это делается на месте).  
∙free — освобождает область памяти, ранее выделенную одной из функций malloc/calloc/realloc.

Для указания размера типа используется оператор sizeof.  

    // создание массива из 1000 int
    int * m = (int *) malloc (1000 * sizeof(int));  // указатель void* из malloc приводится к int*
    m[10] = 10;
    
    // изменение размера массива до 2000
    m = (int *) realloc(m, 2000 * sizeof(int ));    // пытается расширить не двигая, если не получится, то создает новую область под увеличенный объем (освобождая исходную)
    
    // освобождение массива
    free(m);                                        // после этого к m уже нельзя обращаться, ссылка будет не валидна
    
    // создание массива нулей
    m = (int *) calloc (3000,  sizeof(int));
    
    free(m);
    m = 0;  // рекомендуется

## Выделение памяти в стиле C++

∙Язык C++ предоставляет два набора операторов для выделения памяти:
    
    1.new и delete — для одиночных значений,
    2.new [] и delete [] — для массивов.

∙Версия оператора delete должна соответствовать версии оператора new.

    // выделение памяти под один int со значением 5
    int * m = new int (5);                                  // оператор возвращает указатель того типа, который создает (приводить не нужно)
    delete m; // освобождение памяти
    
    // создание массива значений типа int
    m = new  int [1000];
    delete  [] m; // освобождение памяти

### Типичные проблемы при работе с памятью

∙Проблемы производительности: создание переменной на стеке намного “дешевле” выделения для неё динамической памяти.  
∙Проблема фрагментации: выделение большого количества небольших сегментов способствует фрагментации памяти. Решается написанием собственного аллокатора (выделяется сразу большой кусок, в котором программист сам управляет выделением частей под задачи).  
  
∙Утечки памяти:

    // создание массива из 1000 int
    int * m = new int [1000];
    
    // создание массива из 2000 int
    m = new  int [2000];    // утечка памяти от первого new, он будет считаться занятым до конца работы программы и освободить его уже не получиться
    
    // Не вызван delete [] m, утечка памяти

∙Неправильное освобождение памяти.

    int * m1 = new int [1000];
    delete  m1; // должно быть delete [] m1
    
    int * p = new int (0);
    free(p);    // совмещение функций C++ и C
    
    int * q1 = (int *) malloc(sizeof(int ));
    free(q1);
    free(q1);   // двойное удаление (error)
    
    int * q2 = (int *) malloc(sizeof(int ));
    free(q2);
    q2 = 0;     // обнуляем указатель
    free(q2);   // правильно работает для q2 = 0

# Многомерные массивы

## Многомерные встроенные массивы

∙C++позволяет определять многомерные массивы:

    int  m2d [2][3] = { {1, 2, 3}, {4, 5, 6} };
    for( size_t i = 0; i != 2; ++i ) {                  // size_t — специальный целочисленный беззнаковый тип, может вместить в себя размер любого типа в байтах.
        for( size_t j = 0; j != 3; ++j ) {
            cout  << m2d[i][j] << ’ ’;
    }
    cout  << endl;
    }

∙Элементы m2d располагаются в памяти “по строчкам”.  
∙Размерность массивов может быть любой, но на практике редко используют массивы размерности >4.

    int  m4d [2][3][4][5] = {};

## Динамические массивы

∙Для выделения одномерных динамических массивов обычно используется оператор new [].

    int * m1d = new  int [100];

∙Какой тип должен быть у указателя на двумерный динамический массив?  
    
    ∙Пусть m — указатель на двумерный массив типа int.
    ∙Значит m[i][j] имеет тип int (точнее int &).
    ∙m[i][j] ⇔ *(m[i] + j), т.е. тип m[i] — это int *.
    ∙аналогично, m[i] ⇔ *(m + i), т.е. тип m — это int **.

\*) одномерный массив соотв. указателю на int, двумерный - указателю на указатель на int

∙Чему соответствует значение m\[i]? Это адрес строки с номером i.  
∙Чему соответствует значение m? Это адрес массива с указателями на строки.  


### Давайте рассмотрим создание массива 5×4.

    int ** m = new int * [5];               // массив из 5 указателей на 5 массивов длины 4
    for (size_t i = 0; i != 5; ++i)
        m[i] = new int [4];

Выделение и освобождение двумерного массива размера a×b.  
    
    int **  create_array2d(size_t a, size_t b) {
        int ** m = new int *[a];            // массив с указателями на строки
        for (size_t i = 0; i != a; ++i)
            m[i] = new int[b];              // каждый указатель выставляется на созданную строку
        return m;
    }
    
    void  free_array2d(int ** m, size_t a, size_t b) {      // размерность b нам тут (для оператора delete) не нужна, оставлено для единообразия
        for (size_t i = 0; i != a; ++i)
            delete  [] m[i];
        delete  [] m;
    }

При создании массива оператор new вызывается (a+1) раз.  
Память сильно фрагментируется при таком подходе.


## Двумерные массивы: эффективная схема

Массив типа как линейный, но обращаемся с ним как с двухмерным за счет арифметики указателей (а не вычисления индексов, что сделало бы бессмысленным такое представление двумерного массива из-за низкой производительности (??))

### Рассмотрим эффективное создание массива 5×4.

    int ** m = new  int * [5];      // корневой массив указателей на строки
    m[0] = new int[5 * 4];          // все строки массива одним куском в памяти
    for (size_t i = 1; i != 5; ++i)
        m[i] = m[i - 1] + 4;

### Эффективное выделение и освобождение двумерного массива размера a×b.

    int **  create_array2d(size_t a, size_t b) {
        int ** m = new int *[a];
        m[0] = new int[a * b];
        for (size_t i = 1; i != a; ++i)
            m[i] = m[i - 1] + b;
        return m;
    }
    
    void  free_array2d(int ** m, size_t a, size_t b) {  // тут a, b вообще не нужны...
        delete  [] m[0];
        delete  [] m;
    }

При создании массива оператор new вызывается 2 раза.  

\*) аналогично делаются 3-х и более -мерные массивы. Каждый уровнь добавляет по 1-й * к указателю, 1 уровню косвенности, 1 вызову new/delete

Многомерные массивы в C++ можно создавать не только в динамической памяти, но и на стеке. Это делается так же, как и с одномерными массивами:

    int a[5][3];
    cout << a[1][1] << " " << a[1][2] << endl;

Если массив будет проинициализирован сразу при определении, то его внешнюю размерность можно не указывать:

    int a[][3] = { {1, 2, 3}, {4, 5, 6}, {7, 8, 9} };

В примере выше будет создан двумерный массив 3 × 3 (три массива по три элемента).

Многомерный массив на стеке можно проинициализировать ещё и следующим, казалось бы, необычным образом:

    int a[][3] = { 1, 2, 3, 4, 5, 6, 7, 8, 9 };

Дело в том, что многомерные массивы на стеке отличаются от показанных вам на лекции многомерных массивов в динамической памяти — многомерный массив на стеке хранится в непрерывном участке памяти (ему не требуется индексной таблицы с указателями).

Давайте рассмотрим это на примере двумерного массива:

    int a[N][M];

В памяти в начале такого массива хранятся M элементов 0-ой строки, затем M элементов 1-ой строки, затем M элементов 2-ой строки и так далее до строки с номером N - 1 включительно. Другими словами, двумерный массив размера N × M представлен в памяти так же, как и одномерный массив размера NM, поэтому второй вариант инициализации массива эквивалентен первому.

Пользуясь этим знанием, мы теперь можем понять, почему две следующие строки эквивалентны.

    *((int *)a + i * M + j) = 1;

    a[i][j] = 1;

# Строки и ввод-вывод

## Строковые литералы

∙Строки — это массивы символов типа char, заканчивающиеся нулевым символом.

    // массив ’H’, ’e’, ’l’, ’l’, ’o’, ’\0’
    char s[] = "Hello";

∙Строки могут содержать управляющие последовательности:  
    
    1.\n— перевод строки,  
    2.\t— символ табуляции,
    3.\\— символ ’\’,
    4.\"— символ ’"’,
    5.\0— нулевой символ.
    
    cout  << "List:\n\t- C,\n\t- C++.\n";

## Работа со строками в стиле C 

∙Библиотека cstring предлагает множество функций для работы со строками (char *).

    char s1[100] = "Hello";
    cout  << strlen(s1) << endl; // 5

    char s2[] = ", world!";
    strcat(s1, s2);
    
    char s3[6] = {72, 101, 108, 108, 111};
    if (strcmp(s1 , s3) == 0)
        cout  << "s1 == s3" << endl;

∙Работа со строками в стиле C предполагает кропотливую работу с ручным выделением памяти.

Пример реализации strcmp: 

    int strcmp(const char *a, const char *b){
        while ( *a && *b && *a == *b ) ++a, ++b;
        return *a - *b;
    }

## Работа со строками в стиле C++ 

Библиотека string предлагает обёртку над строками, которая позволяет упростить все операции со строками.

    #include  <string >
    using  namespace  std;
    
    int  main() {
        string  s1 = "Hello";
        cout  << s1.size() << endl; // 5
        
        string  s2 = ", world!";
        s1 = s1 + s2;
        if (s1 == s2)
            cout  << "s1 == s2" << endl;
        return  0;
    }

# Ввод-вывод в стиле C 

∙Библиотека cstdio предлагает функции для работы со стандартным вводом-выводом.  

∙Для вывода используется функция printf:

    #include  <cstdio >
    int  main() {
        int h = 20, m = 14;
        printf("Time: %d:%d\n", h, m);
        printf("It’s %.2f hours  to  midnight\n",
              ((24 - h) * 60.0 - m) / 60);
        return  0;
    }

∙Для ввода используется функция scanf:

    #include  <cstdio >
    int  main() {
        int a = 0, b = 0;
        printf("Enter a and b: ");
        scanf("%d %d", &a, &b);
        printf("a + b = %d\n", (a + b));
        return  0;
    }

∙Ввод-вывод в стиле C достаточно сложен и небезопасен (типы аргументов не проверяются).

## Ввод-вывод в стиле C++

∙В C++ ввод-вывод реализуется через библиотеку iostream.
    
    #include  <string >
    #include  <iostream >
    using  namespace  std;

    int  main() {
        string  name;
        cout  << "Enter  your  name: ";

        cin  >> name;                       // считывается слово
        cout  << "Hi , " << name  << endl;
        return  0;
    }
∙Реализация ввода-вывода в стиле C++ типобезопасна (тип вводимого значения определяет по типу переменно, в которую оно пишется).

# Работа с файлами в стиле C++

∙Библиотека fstream обеспечивает работу с файлами.

    #include  <string >
    #include  <fstream >
    using  namespace  std;
    
    int  main() {
        string  name;
        ifstream  input("input.txt");
        input  >> name;
        
        ofstream  output("output.txt");
        output  << "Hi , " << name  << endl;
        return  0;
    }

∙Файлы закроются при выходе из функции.

## Как можно перенаправить стандартный поток ввода так чтобы он читал из  файла.

Первый способ относится к функциям С (можно поместить в качестве первой инструкции в функции main):

    freopen("input.txt", "r", stdin); // перенаправление стандартного потока в стиле C

Второй способ использует С++:

    #include <iostream>
    #include <fstream>

    std::streambuf *stdin_bcp = std::cin.rdbuf();   // сохраняем старый поток stdin
    std::ifstream ifs("input.txt");                 // Создаём поток для чтения из файла "input.txt"
    std::cin.rdbuf(ifs.rdbuf());                    // Перенаправили стандартный ввод на ввод из файла

    //..... Ваш код ........

    std::cin.rdbuf(stdin_bcp);                      // восстанавливаем поток stdin по умолчанию
    ifs.close();                                    // закрываем поток чтения из файла.
